In [1]:
import numpy as np
import pandas as pd
import os

## Funções e Configurações

In [2]:
# this dictionary contains the configuration necessary to divide each dataset used in the work, the script is made
# for the conll type of datasets, so if you want to add datasets to divide, make sure to add it here
gen_config = {
    'leNER':{
        'base_dir': "../Base de Dados/leNER/",
        'load_dir': "documents/allDoc/",
        'save_dir': "divisions/iterative/",
        'col_sep' : ' ',
        'file_type': '.conll',
    },
    'UlyssesNER-BR':{
        'base_dir': "../Base de Dados/UlyssesNER-BR/",
        'load_dir': "PL_Corpus_byTypes_conll/",
        'save_dir': "divisions/iterative/",
        'col_sep' : ' ',
        'file_type': '.conll',
    },
    'Harem-second':{
        'base_dir': "../Base de Dados/HAREM/",
        'load_dir': "SecondHarem/documents",
        'save_dir': "divisions/second/iterative/",
        'col_sep' : '\t',
        'file_type': '.conll',
    }
}

In [3]:
def get_sentencas(corpus, coll_sep):
        sentenca = []
        for line in corpus:
            if line == '\n':
                if sentenca:
                    yield sentenca
                    sentenca = []
            else:
                
                sentenca.append(line.strip('\n').split(coll_sep))
        if sentenca:
            yield sentenca

In [4]:
def read_dataset(filepath,coll_sep):
    data_por_sentenca = []
    with open(filepath, "r") as corpus:
        data_por_sentenca += list(get_sentencas(corpus,coll_sep))
    
    return data_por_sentenca

In [5]:
def get_data(config, selective = False, fileName = ""):
    load_dir = config['base_dir'] + config['load_dir']
    sep = config['col_sep']
    if selective:
        fp = load_dir + fileName
        return read_dataset(filepath=fp, coll_sep=sep)
    else:
        filepaths = []
        for file in os.listdir(load_dir):
            if file.endswith(config['file_type']):
                filepaths.append(load_dir + file)
        
        data_por_sentenca = []
        for filepath in filepaths:
            print(filepath+"\n")
            data_por_sentenca += read_dataset(filepath=filepath, coll_sep=sep)
        
        return data_por_sentenca

In [6]:
def split_token_label(data):
    return list(map(list,zip(*data)))

## Carregando DataSet e preparando dados

In [7]:
# dataset = 'leNER'
# dataset = 'Paramopama'
# dataset = 'Harem-mini'
dataset = 'Harem-second'
# dataset = 'UlyssesNER-BR'

config = gen_config[dataset]

filepaths = []

files = os.listdir(config['base_dir'] + config['load_dir'])
print(files)
for file in files:
    # if file.endswith('.conll'):
    filepaths.append(config['base_dir'] + config['load_dir'] + file)

# data_por_sentenca = get_data(config=config)

# data_por_sentenca = get_data(config=config, selective=True, fileName='corpus_Paramopama.txt')
# data_por_sentenca = get_data(config=config, selective=True, fileName='haremMini-total.colnn')
# data_por_sentenca = get_data(config=config, selective=True, fileName='primeiroHAREM-total.colnn')
folder_path = '../Base de Dados/HAREM/SecondHarem/documents'
data_por_sentenca_por_doc = {}

# Listar arquivos dentro da pasta
if os.path.exists(folder_path):
    for file in os.listdir(folder_path):
        filepath = os.path.join(folder_path, file)
        
        # Verificar se é um arquivo
        if os.path.isfile(filepath):
            data_por_sentenca_por_doc[file] = read_dataset(filepath, coll_sep=config['col_sep'])
else:
    print(f"O diretório {folder_path} não existe. Verifique o caminho.")

# Exibir resultado
total_documentos = len(data_por_sentenca_por_doc)
print("Total de documentos carregados:", total_documentos)


['hub-93257', 'hub-74208', 'hub-49343', 'ric-30606', 'ven-098', 'hub-4171', 'hub-16632', 'H2-Ren_2001_6414', 'hub-96408', 'hub-31642', 'hub-39869', 'hub-28306', 'rica-474415', 'hub-66746', 'hub-15425', 'hub-22322', 'ric-81275', 'hub-94890', 'ric-92221', 'hub-30518', 'hub-21881', 'hub-64260', 'hub-83689', 'hub-37295', 'aa33715', 'Ntyr-78', 'ric-46913', 'bob-14949', 'ric-19806', 'aa40383', 'H2-Ren_2003_6465', 'dav-52390', 'dav-844651', 'ric-47441', 'aa56088', 'hub-67792', 'hub-77889', 'DR-PT5', 'dav-522210', 'ric-39610', 'H2-bbb', 'ric-33939', 'H2-Ert75', 'ric-54609', 'dav-100932', 'ric-85133', 'hub-2312', 'hub-822', 'exp-76321', 'hub-398', 'H2-gt654', 'hub-60382', 'hub-66526', 'aa46996', 'hub-83020', 'ric-46546', 'gtqqq', 'wpt-1019468604217317242', 'ric-74122', 'hub-2701', 'aa25258', 'hub-60905', 'hub-65266', 'hub-5080', 'hub-47914', 'Ytr433', 'wpt-101306633536471708', 'H2-EFin', 'bob-37600', 'wpt-101789548982731354', 'aa94781', 'wpt-1016797200209519613', 'hub-44595', 'hjlll', 'hub-5146

In [8]:
# data_por_sentenca_por_doc['HAREMdoc_aa64686'][0]
# len(data_por_sentenca_por_doc['HAREMdoc_aa64686'])
# data_por_sentenca_por_doc[list(data_por_sentenca_por_doc.keys())[0]]
# d = list(data_por_sentenca_por_doc.values())[0][10][-1]
# list(map(list,zip(d)))
# # d

In [9]:
from collections import Counter
labels_por_doc = {}

for docName, docData in data_por_sentenca_por_doc.items():
    allDocData = []
    
    for sentence in docData:
        allDocData += [i for i in sentence]
    
    docTokens, docLabels = split_token_label(allDocData)
    
    
    labels_por_doc[docName] = docLabels

    labels_count = Counter(docLabels)
    
    # Armazenar a contagem de rótulos para o documento atual
    labels_por_doc[docName] = dict(labels_count)

# Exibir as contagens de rótulos por documento
for doc, labels_count in labels_por_doc.items():
    print(f"Documento: {doc}")
    print("Contagem de rótulos:", labels_count)
    print()

Documento: hub-93257
Contagem de rótulos: {'B-PESSOA': 23, 'I-PESSOA': 20, 'O': 398, 'B-ACONTECIMENTO': 2, 'I-ACONTECIMENTO': 4, 'B-LOCAL': 12, 'B-ORGANIZACAO': 6, 'I-ORGANIZACAO': 2, 'B-OBRA': 1, 'I-OBRA': 1, 'B-TEMPO': 6, 'I-TEMPO': 22, 'I-LOCAL': 1}

Documento: hub-74208
Contagem de rótulos: {'O': 192, 'B-COISA': 8, 'B-TEMPO': 3, 'I-TEMPO': 7, 'B-ABSTRACCAO': 2, 'I-ABSTRACCAO': 1, 'B-LOCAL': 2, 'I-LOCAL': 3, 'B-VALOR': 7, 'I-VALOR': 7, 'B-PESSOA': 3, 'I-PESSOA': 3, 'I-COISA': 4, 'B-ORGANIZACAO': 1}

Documento: hub-49343
Contagem de rótulos: {'B-PESSOA': 14, 'I-PESSOA': 28, 'O': 334, 'B-TEMPO': 15, 'I-TEMPO': 26, 'B-LOCAL': 33, 'B-ORGANIZACAO': 9, 'I-ORGANIZACAO': 7, 'I-LOCAL': 17, 'B-VALOR': 1, 'I-VALOR': 1, 'B-ABSTRACCAO': 1, 'I-ABSTRACCAO': 4}

Documento: ric-30606
Contagem de rótulos: {'O': 867, 'B-TEMPO': 5, 'I-TEMPO': 5, 'B-LOCAL': 4, 'I-LOCAL': 3, 'B-ACONTECIMENTO': 1, 'I-ACONTECIMENTO': 2, 'B-OBRA': 10, 'I-OBRA': 18, 'B-COISA': 6, 'B-PESSOA': 12, 'I-PESSOA': 10, 'I-COISA': 3,

In [10]:
# labels_por_doc['HAREMdoc_aa64686'][-12]
# len(labels_por_doc['HAREMdoc_aa64686'])

In [11]:
from collections import Counter

# Inicializar um dicionário para armazenar contagens de rótulos por documento
# antes e depois de simplificar.
labels_por_doc = {}
labels_por_doc_simplificado = {}

for docName, docData in data_por_sentenca_por_doc.items():
    allDocData = []
    
    # Concatenar todas as sentenças de um documento em uma lista
    for sentence in docData:
        allDocData += [i for i in sentence]
    
    # Separar tokens e rótulos
    docTokens, docLabels = split_token_label(allDocData)
    
    # Contagem de rótulos com os prefixos (original)
    labels_count_original = Counter(docLabels)
    labels_por_doc[docName] = dict(labels_count_original)
    
    # Extração da última parte do rótulo (ex. 'B-PER' vira 'PER')
    labels_simplificados = [lbl.split('-')[-1] for lbl in docLabels]
    labels_count_simplificado = Counter(labels_simplificados)
    labels_por_doc_simplificado[docName] = dict(labels_count_simplificado)

# Exibir a contagem original e simplificada para comparação
for doc in labels_por_doc.keys():
    print(f"Documento: {doc}")
    print("Contagem de rótulos (original):", labels_por_doc[doc])
    print("Contagem de rótulos (simplificado):", labels_por_doc_simplificado[doc])
    print()

# Exibir o total de documentos
print("Total de documentos:", len(labels_por_doc))



Documento: hub-93257
Contagem de rótulos (original): {'B-PESSOA': 23, 'I-PESSOA': 20, 'O': 398, 'B-ACONTECIMENTO': 2, 'I-ACONTECIMENTO': 4, 'B-LOCAL': 12, 'B-ORGANIZACAO': 6, 'I-ORGANIZACAO': 2, 'B-OBRA': 1, 'I-OBRA': 1, 'B-TEMPO': 6, 'I-TEMPO': 22, 'I-LOCAL': 1}
Contagem de rótulos (simplificado): {'PESSOA': 43, 'O': 398, 'ACONTECIMENTO': 6, 'LOCAL': 13, 'ORGANIZACAO': 8, 'OBRA': 2, 'TEMPO': 28}

Documento: hub-74208
Contagem de rótulos (original): {'O': 192, 'B-COISA': 8, 'B-TEMPO': 3, 'I-TEMPO': 7, 'B-ABSTRACCAO': 2, 'I-ABSTRACCAO': 1, 'B-LOCAL': 2, 'I-LOCAL': 3, 'B-VALOR': 7, 'I-VALOR': 7, 'B-PESSOA': 3, 'I-PESSOA': 3, 'I-COISA': 4, 'B-ORGANIZACAO': 1}
Contagem de rótulos (simplificado): {'O': 192, 'COISA': 12, 'TEMPO': 10, 'ABSTRACCAO': 3, 'LOCAL': 5, 'VALOR': 14, 'PESSOA': 6, 'ORGANIZACAO': 1}

Documento: hub-49343
Contagem de rótulos (original): {'B-PESSOA': 14, 'I-PESSOA': 28, 'O': 334, 'B-TEMPO': 15, 'I-TEMPO': 26, 'B-LOCAL': 33, 'B-ORGANIZACAO': 9, 'I-ORGANIZACAO': 7, 'I-LOCA

In [12]:
# labels_por_doc['HAREMdoc_aa64686'][-12]
# len(labels_por_doc['HAREMdoc_aa64686'])

In [14]:
lbl_list = []
for lbl in labels_por_doc.values():
    lbl_list.extend(lbl)  # Adiciona os rótulos de cada documento à lista

# Filtrando 'B-OUTRO' e 'I-OUTRO'
lbl_list = [label for label in lbl_list if label not in []]

# Obtendo os rótulos únicos
unique_lbl_list = list(set(lbl_list))

# Exibindo a lista de rótulos únicos
print(unique_lbl_list)




['I-ORGANIZACAO', 'I-COISA', 'B-OUTRO', 'B-LOCAL', 'I-TEMPO', 'I-ACONTECIMENTO', 'B-ACONTECIMENTO', 'B-COISA', 'O', 'I-PESSOA', 'B-PESSOA', 'B-TEMPO', 'B-ABSTRACCAO', 'B-OBRA', 'I-ABSTRACCAO', 'I-OBRA', 'B-ORGANIZACAO', 'I-VALOR', 'I-OUTRO', 'I-LOCAL', 'B-VALOR']


In [15]:
unique_lbl_list.pop(unique_lbl_list.index('O'))
unique_lbl_list

['I-ORGANIZACAO',
 'I-COISA',
 'B-OUTRO',
 'B-LOCAL',
 'I-TEMPO',
 'I-ACONTECIMENTO',
 'B-ACONTECIMENTO',
 'B-COISA',
 'I-PESSOA',
 'B-PESSOA',
 'B-TEMPO',
 'B-ABSTRACCAO',
 'B-OBRA',
 'I-ABSTRACCAO',
 'I-OBRA',
 'B-ORGANIZACAO',
 'I-VALOR',
 'I-OUTRO',
 'I-LOCAL',
 'B-VALOR']

In [16]:
unique_lbl_list = sorted(unique_lbl_list)
unique_lbl_list

['B-ABSTRACCAO',
 'B-ACONTECIMENTO',
 'B-COISA',
 'B-LOCAL',
 'B-OBRA',
 'B-ORGANIZACAO',
 'B-OUTRO',
 'B-PESSOA',
 'B-TEMPO',
 'B-VALOR',
 'I-ABSTRACCAO',
 'I-ACONTECIMENTO',
 'I-COISA',
 'I-LOCAL',
 'I-OBRA',
 'I-ORGANIZACAO',
 'I-OUTRO',
 'I-PESSOA',
 'I-TEMPO',
 'I-VALOR']

In [17]:
#test_l = ['LOCAL', 'ORGANIZACAO', 'PESSOA', 'TEMPO', 'LOCAL', 'ORGANIZACAO', 'PESSOA', 'TEMPO', 'LOCAL', 'ORGANIZACAO', 'PESSOA', 'TEMPO', 'TEMPO', 'TEMPO']
#[test_l.count(x) for x in unique_lbl_list]

In [18]:
from collections import Counter

initial_set = []
for s_idx, s_labels in enumerate(list(labels_por_doc.values())):
    # Contagem eficiente dos rótulos usando Counter
    label_count = Counter(s_labels)
    
    # Adicionando o índice do documento e as contagens para cada rótulo em unique_lbl_list
    initial_set.append([s_idx] + [label_count.get(x, 0) for x in unique_lbl_list])

# Verificando o tamanho de initial_set (número de documentos)
print(len(initial_set))


129


In [18]:
# initial_set[:, 1:]

In [19]:
# Adicionando a linha final com os totais de cada label (soma dos rótulos)
initial_set.append(['-'] + [sum(x) for x in zip(*(row[1:] for row in initial_set))])

# Agora vamos filtrar a matriz para manter apenas as 7 primeiras colunas
# (assumindo que as 7 primeiras colunas correspondem aos rótulos -B)
filtered_initial_set = []

for row in initial_set:
    # Mantendo apenas as 7 primeiras colunas (a primeira coluna é '-')
    filtered_row = row[:11]  # Considerando que as primeiras 7 colunas são os rótulos -B
    filtered_initial_set.append(filtered_row)

# Exibindo a matriz resultante filtrada
for row in filtered_initial_set:
    print(row)


[0, 0, 2, 0, 12, 1, 6, 0, 23, 6, 0]
[1, 2, 0, 8, 2, 0, 1, 0, 3, 3, 7]
[2, 1, 0, 0, 33, 0, 9, 0, 14, 15, 1]
[3, 1, 1, 6, 4, 10, 2, 0, 12, 5, 4]
[4, 3, 0, 1, 7, 0, 8, 0, 14, 4, 2]
[5, 2, 3, 0, 1, 0, 4, 0, 2, 8, 4]
[6, 0, 0, 0, 2, 0, 0, 0, 5, 0, 0]
[7, 0, 0, 3, 5, 0, 2, 0, 3, 2, 2]
[8, 1, 8, 1, 6, 0, 0, 0, 24, 1, 11]
[9, 0, 2, 0, 8, 0, 5, 0, 8, 6, 0]
[10, 0, 2, 0, 9, 0, 12, 0, 12, 5, 0]
[11, 1, 0, 10, 1, 2, 4, 0, 0, 3, 0]
[12, 1, 1, 0, 11, 3, 18, 0, 20, 13, 1]
[13, 0, 0, 0, 1, 2, 15, 0, 11, 7, 1]
[14, 3, 1, 0, 3, 10, 10, 0, 4, 13, 1]
[15, 0, 0, 0, 1, 1, 3, 0, 9, 9, 0]
[16, 9, 0, 7, 3, 6, 0, 0, 20, 6, 0]
[17, 0, 5, 0, 14, 1, 0, 0, 5, 15, 4]
[18, 1, 3, 0, 11, 0, 0, 1, 19, 2, 7]
[19, 1, 1, 9, 3, 0, 0, 0, 2, 3, 0]
[20, 0, 0, 0, 0, 0, 17, 0, 2, 5, 1]
[21, 0, 1, 2, 7, 4, 2, 1, 1, 2, 0]
[22, 2, 0, 0, 17, 1, 1, 0, 18, 7, 3]
[23, 0, 2, 1, 6, 0, 0, 0, 4, 6, 1]
[24, 3, 0, 1, 0, 0, 13, 0, 15, 3, 2]
[25, 0, 7, 0, 20, 5, 5, 0, 43, 22, 16]
[26, 16, 0, 1, 4, 0, 0, 1, 3, 6, 0]
[27, 0, 3, 0, 20, 3, 2, 0, 2

In [20]:
# list(labels_por_doc.keys())

In [21]:
#initial_set[-1] = initial_set[-1][:7]  # Isso mantém as 7 primeiras colunas, incluindo '-'

# Exibindo a última linha após a atualização
print(filtered_initial_set[-1])


['-', 382, 337, 345, 1453, 497, 1080, 81, 2091, 1199, 352]


In [22]:
len(filtered_initial_set)

130

In [23]:
number_divisions = 10
result = [x / number_divisions for x in filtered_initial_set[-1][1:]]
print(result)


[38.2, 33.7, 34.5, 145.3, 49.7, 108.0, 8.1, 209.1, 119.9, 35.2]


In [24]:

sub_set = [[['-'] + [x/number_divisions for x in filtered_initial_set[-1][1:]]] for i in range(number_divisions)]
sub_set

[[['-', 38.2, 33.7, 34.5, 145.3, 49.7, 108.0, 8.1, 209.1, 119.9, 35.2]],
 [['-', 38.2, 33.7, 34.5, 145.3, 49.7, 108.0, 8.1, 209.1, 119.9, 35.2]],
 [['-', 38.2, 33.7, 34.5, 145.3, 49.7, 108.0, 8.1, 209.1, 119.9, 35.2]],
 [['-', 38.2, 33.7, 34.5, 145.3, 49.7, 108.0, 8.1, 209.1, 119.9, 35.2]],
 [['-', 38.2, 33.7, 34.5, 145.3, 49.7, 108.0, 8.1, 209.1, 119.9, 35.2]],
 [['-', 38.2, 33.7, 34.5, 145.3, 49.7, 108.0, 8.1, 209.1, 119.9, 35.2]],
 [['-', 38.2, 33.7, 34.5, 145.3, 49.7, 108.0, 8.1, 209.1, 119.9, 35.2]],
 [['-', 38.2, 33.7, 34.5, 145.3, 49.7, 108.0, 8.1, 209.1, 119.9, 35.2]],
 [['-', 38.2, 33.7, 34.5, 145.3, 49.7, 108.0, 8.1, 209.1, 119.9, 35.2]],
 [['-', 38.2, 33.7, 34.5, 145.3, 49.7, 108.0, 8.1, 209.1, 119.9, 35.2]]]

## 
ITERATIVE STRATIFICATION

- while there're still sentences in the initial_set
    - update number of each label in the initial_set
    - select label in initial_set that has fewest (at least 1) example
    - then for each sentences that contains this label,
        - find the subset that desires it more and add it to it
        - update the desired number of each label for the subset

In [25]:
# initial_set_l = [x for x in initial_set[:-1] if x[2] > 0]
# len(initial_set_l)
# 12168 - 2734
# for sentence in initial_set_l:
#     initial_set.pop(initial_set.index(sentence))
    

In [26]:
# len(initial_set)
# initial_set[-1]
# initial_set[-1] = ['-'] + [ sum(x) for x in zip(*(row[1:] for row in initial_set[:-1])) ]
# initial_set[-1]

In [27]:
while len(filtered_initial_set) > 1:
    print(f"\nTamanho atual de filtered_initial_set: {len(filtered_initial_set)}")
    contador = 0

    try:
        # Identificar o índice da label com menor quantidade
        min_lbl_idx = filtered_initial_set[-1].index(min((i for i in filtered_initial_set[-1][1:] if i > 0)))
        print(f"Label com menor quantidade: {min_lbl_idx} (quantidade: {filtered_initial_set[-1][min_lbl_idx]})")

        # Filtrar as sentenças relevantes
        initial_set_l = [x for x in filtered_initial_set[:-1] if x[min_lbl_idx] > 0]

        # Ordenar as sentenças pela quantidade da palavra desejada (min_lbl_idx), em ordem decrescente
        initial_set_l.sort(key=lambda x: x[min_lbl_idx], reverse=True)

        print(f"\nSentenças ordenadas pela quantidade da palavra desejada (label {min_lbl_idx}):")
        for idx, item in enumerate(initial_set_l):
            print(f"  {idx + 1}: {item}")

        # Inicializar o índice das partições
        partition_order = list(range(10))  # 10 partições

        for sentence in initial_set_l:
            partition_found = False

            # Ordenar as partições pela quantidade de desejo da min_lbl_idx, em ordem decrescente
            sorted_partitions = sorted(partition_order, key=lambda idx: sub_set[idx][0][min_lbl_idx], reverse=True)

            for partition_idx in sorted_partitions:
                current_partition_desires = sub_set[partition_idx][0][1:]
                potential_desires = [
                    current - count
                    for current, count in zip(current_partition_desires, sentence[1:])
                ]

                # Verificar se algum desejo se torna negativo
                if all(potential >= 0 for potential in potential_desires):
                    # Se a partição permitir a adição sem causar desejos negativos, adicione a sentença
                    sub_set[partition_idx].append(sentence)
                    filtered_initial_set.pop(filtered_initial_set.index(sentence))

                    # Atualizar o desejo de cada label na partição
                    sub_set[partition_idx][0][1:] = [
                        current - count for current, count in zip(sub_set[partition_idx][0][1:], sentence[1:])
                    ]

                    # Atualizar os valores em filtered_initial_set
                    filtered_initial_set[-1] = ['-'] + [sum(x) for x in zip(*(row[1:] for row in filtered_initial_set[:-1]))]

                    # Imprimir status atualizado
                    total_entities_in_partition = sum(
                        sum(doc[1:]) for doc in sub_set[partition_idx][1:]
                    )
                    print(f"\nSentença adicionada: {sentence}")
                    print(f"Partição {partition_idx} agora tem {total_entities_in_partition} entidades.")

                    # Imprimir o desejo de cada partição
                    print("\nDesejo de cada partição após adicionar a sentença:")
                    for i, partition in enumerate(sub_set):
                        print(f"Partição {i}: {partition[0][1:]}")

                    partition_found = True
                    break  # Sentença adicionada, pode sair do loop

            if not partition_found:
                # Caso a sentença não tenha sido alocada sem negativos, vamos escolher a partição com o menor número de entidades negativas
                best_negative_count = float('inf')  # Maior soma de negativos
                best_negative_sum = float('inf')   # Soma dos negativos
                best_partition_idx = None

                for partition_idx in sorted_partitions:
                    current_partition_desires = sub_set[partition_idx][0][1:]
                    potential_desires = [
                        current - count
                        for current, count in zip(current_partition_desires, sentence[1:])
                    ]

                    # Contar quantos desejos se tornam negativos
                    negative_count = sum(1 for desire in potential_desires if desire < 0)

                    # Calcular a soma dos valores negativos
                    negative_sum = sum(1 for desire in potential_desires if desire < 0)

                    # Se o número de negativos for menor, ou se for igual e a soma dos negativos for maior, escolher esta partição
                    if negative_count < best_negative_count or (negative_count == best_negative_count and negative_sum > best_negative_sum):
                        best_negative_count = negative_count
                        best_negative_sum = negative_sum
                        best_partition_idx = partition_idx

                if best_partition_idx is not None:
                    # Adicionar a sentença na partição com o menor impacto negativo
                    sub_set[best_partition_idx].append(sentence)
                    sub_set[best_partition_idx][0][1:] = [
                        current - count for current, count in zip(sub_set[best_partition_idx][0][1:], sentence[1:])
                    ]
                    filtered_initial_set.pop(filtered_initial_set.index(sentence))

                    # Atualizar os valores em filtered_initial_set
                    filtered_initial_set[-1] = ['-'] + [sum(x) for x in zip(*(row[1:] for row in filtered_initial_set[:-1]))]

                    # Imprimir status atualizado
                    total_entities_in_partition = sum(
                        sum(doc[1:]) for doc in sub_set[best_partition_idx][1:]
                    )
                    print(f"\nSentença adicionada (com negativos): {sentence}")
                    print(f"Partição {best_partition_idx} agora tem {total_entities_in_partition} entidades.")

                    # Imprimir o desejo de cada partição
                    print("\nDesejo de cada partição após adicionar a sentença:")
                    for i, partition in enumerate(sub_set):
                        print(f"Partição {i}: {partition[0][1:]}")

            contador += 1

        # Recalcular a label com menor quantidade após a alocação das sentenças
        min_lbl_idx = filtered_initial_set[-1].index(min((i for i in filtered_initial_set[-1][1:] if i > 0)))

    except ValueError:
        # Caso ValueError, processar as sentenças restantes sem basear-se na label
        initial_set_l = filtered_initial_set[:-1]

        print(f"\nConteúdo de initial_set_l (ValueError handler, {len(initial_set_l)} elementos):")
        for idx, item in enumerate(initial_set_l):
            print(f"  {idx + 1}: {item}")

        for sentence in initial_set_l:
            # Verificar os tamanhos das partições
            partition_sizes = [sum(sum(doc[1:]) for doc in partition[1:]) for partition in sub_set]
            partitions_below_limit = [i for i, size in enumerate(partition_sizes) if size < 1225]

            # Escolher a partição com menor valor total
            if partitions_below_limit:
                target_partition_idx = min(partitions_below_limit, key=lambda i: partition_sizes[i])
            else:
                target_partition_idx = partition_sizes.index(min(partition_sizes))

            # Adicionar a sentença na partição escolhida
            sub_set[target_partition_idx].append(sentence)
            filtered_initial_set.pop(filtered_initial_set.index(sentence))

            # Atualizar os valores em filtered_initial_set
            filtered_initial_set[-1] = ['-'] + [sum(x) for x in zip(*(row[1:] for row in filtered_initial_set[:-1]))]

            # Imprimir status atualizado
            total_entities_in_partition = partition_sizes[target_partition_idx] + sum(sentence[1:])
            print(f"\nSentença adicionada: {sentence}")
            print(f"Partição {target_partition_idx} agora tem {total_entities_in_partition} entidades.")

    print(f"\nEstado atualizado de filtered_initial_set: {filtered_initial_set[-1]}")
    print(f"Total de sentenças processadas nesta iteração: {contador}")



Tamanho atual de filtered_initial_set: 130
Label com menor quantidade: 7 (quantidade: 81)

Sentenças ordenadas pela quantidade da palavra desejada (label 7):
  1: [38, 9, 31, 4, 60, 27, 30, 13, 56, 26, 0]
  2: [74, 0, 0, 0, 7, 0, 19, 8, 1, 24, 36]
  3: [91, 10, 0, 5, 47, 12, 18, 7, 57, 50, 5]
  4: [31, 4, 32, 16, 42, 17, 20, 5, 63, 35, 0]
  5: [66, 14, 14, 5, 32, 26, 100, 5, 22, 20, 7]
  6: [44, 2, 8, 7, 57, 7, 32, 4, 56, 41, 7]
  7: [72, 0, 8, 0, 6, 0, 0, 4, 18, 5, 1]
  8: [32, 5, 16, 12, 51, 4, 7, 3, 40, 19, 1]
  9: [57, 0, 0, 0, 3, 23, 15, 3, 0, 7, 0]
  10: [86, 5, 20, 6, 42, 13, 19, 3, 69, 26, 0]
  11: [52, 1, 0, 0, 14, 1, 5, 2, 18, 6, 1]
  12: [58, 8, 1, 20, 5, 23, 6, 2, 80, 12, 0]
  13: [104, 48, 0, 4, 23, 1, 0, 2, 4, 10, 0]
  14: [108, 4, 4, 0, 14, 1, 16, 2, 14, 11, 1]
  15: [123, 0, 5, 13, 2, 0, 0, 2, 21, 7, 11]
  16: [18, 1, 3, 0, 11, 0, 0, 1, 19, 2, 7]
  17: [21, 0, 1, 2, 7, 4, 2, 1, 1, 2, 0]
  18: [26, 16, 0, 1, 4, 0, 0, 1, 3, 6, 0]
  19: [29, 0, 5, 0, 6, 1, 7, 1, 43, 18, 1

In [28]:
filtered_initial_set

[['-']]

In [29]:
for sub in sub_set:
    print('tam: {} - lbs:{}'.format(len(sub[1:]), sub[0]))

tam: 11 - lbs:['-', 6.200000000000003, 0.7000000000000028, 5.5, -22.69999999999999, -26.299999999999997, 11.0, -4.9, 72.1, 15.900000000000006, 4.200000000000003]
tam: 12 - lbs:['-', 10.200000000000003, 9.700000000000003, -22.5, -35.69999999999999, 6.700000000000003, 1.0, 0.09999999999999964, 32.099999999999994, 12.900000000000006, -30.799999999999997]
tam: 12 - lbs:['-', 10.200000000000003, 0.7000000000000028, -8.5, 6.300000000000011, 1.7000000000000028, -4.0, 0.09999999999999964, 1.0999999999999943, -20.099999999999994, 8.200000000000003]
tam: 14 - lbs:['-', 9.200000000000003, -4.299999999999997, 0.5, 1.3000000000000114, 1.7000000000000028, -6.0, 0.09999999999999964, 21.099999999999994, -1.0999999999999943, 11.200000000000003]
tam: 16 - lbs:['-', 0.20000000000000284, 0.7000000000000028, 0.5, 9.300000000000011, 0.7000000000000028, -59.0, 0.09999999999999964, 9.099999999999994, 2.9000000000000057, -18.799999999999997]
tam: 14 - lbs:['-', 9.200000000000003, -0.29999999999999716, 15.5, -1

In [30]:
for sub in sub_set:
    
    print (sub)
    

[['-', 6.200000000000003, 0.7000000000000028, 5.5, -22.69999999999999, -26.299999999999997, 11.0, -4.9, 72.1, 15.900000000000006, 4.200000000000003], [38, 9, 31, 4, 60, 27, 30, 13, 56, 26, 0], [68, 19, 2, 0, 2, 26, 27, 0, 4, 2, 1], [85, 2, 0, 1, 2, 18, 1, 0, 18, 12, 1], [2, 1, 0, 0, 33, 0, 9, 0, 14, 15, 1], [126, 1, 0, 0, 9, 1, 1, 0, 0, 7, 1], [62, 0, 0, 0, 3, 2, 15, 0, 15, 8, 0], [15, 0, 0, 0, 1, 1, 3, 0, 9, 9, 0], [95, 0, 0, 0, 11, 1, 2, 0, 3, 4, 5], [114, 0, 0, 11, 20, 0, 3, 0, 17, 9, 3], [61, 0, 0, 7, 0, 0, 4, 0, 0, 8, 8], [75, 0, 0, 6, 27, 0, 2, 0, 1, 4, 11]]
[['-', 10.200000000000003, 9.700000000000003, -22.5, -35.69999999999999, 6.700000000000003, 1.0, 0.09999999999999964, 32.099999999999994, 12.900000000000006, -30.799999999999997], [74, 0, 0, 0, 7, 0, 19, 8, 1, 24, 36], [34, 2, 4, 0, 7, 21, 40, 0, 39, 9, 3], [118, 1, 4, 1, 3, 3, 3, 0, 4, 6, 1], [98, 9, 3, 0, 14, 3, 3, 0, 9, 9, 1], [102, 0, 3, 0, 22, 9, 0, 0, 6, 12, 8], [106, 1, 3, 0, 21, 0, 37, 0, 18, 5, 0], [120, 0, 3, 0, 5, 

In [31]:
len(sub_set[-1])

15

In [32]:
# Iterando sobre cada subconjunto em sub_set
for idx, s_set in enumerate(sub_set):
    # Construindo o caminho do arquivo para o subconjunto, com extensão .txt
    file_path = f"../Base de Dados/HAREM/divisions/second/iterative/division_{idx}.txt"
    
    # Abrindo o arquivo para o subconjunto
    with open(file_path, 'w+', encoding='utf-8') as fp:
        # Iterando sobre cada documento no subconjunto, excluindo a linha de contagens (assumindo que está na posição 0)
        for set_doc in s_set[1:]:
            # Obter o nome do documento usando o índice do documento
            doc_name = list(data_por_sentenca_por_doc.keys())[set_doc[0]]
            doc_sentences = data_por_sentenca_por_doc[doc_name]

            # Escrever cada sentença do documento no arquivo
            for sentence in doc_sentences:
                for tk_class in sentence:
                    # Gravar cada elemento da sentença, com separador adequado
                    fp.write(config['col_sep'].join(map(str, tk_class)) + '\n')
                # Quebra de linha entre sentenças
                fp.write('\n')
    
    print(f"Subconjunto {idx} salvo com sucesso em {file_path}")



Subconjunto 0 salvo com sucesso em ../Base de Dados/HAREM/divisions/second/iterative/division_0.txt
Subconjunto 1 salvo com sucesso em ../Base de Dados/HAREM/divisions/second/iterative/division_1.txt
Subconjunto 2 salvo com sucesso em ../Base de Dados/HAREM/divisions/second/iterative/division_2.txt
Subconjunto 3 salvo com sucesso em ../Base de Dados/HAREM/divisions/second/iterative/division_3.txt
Subconjunto 4 salvo com sucesso em ../Base de Dados/HAREM/divisions/second/iterative/division_4.txt
Subconjunto 5 salvo com sucesso em ../Base de Dados/HAREM/divisions/second/iterative/division_5.txt
Subconjunto 6 salvo com sucesso em ../Base de Dados/HAREM/divisions/second/iterative/division_6.txt
Subconjunto 7 salvo com sucesso em ../Base de Dados/HAREM/divisions/second/iterative/division_7.txt
Subconjunto 8 salvo com sucesso em ../Base de Dados/HAREM/divisions/second/iterative/division_8.txt
Subconjunto 9 salvo com sucesso em ../Base de Dados/HAREM/divisions/second/iterative/division_9.txt


In [33]:
import os
from collections import Counter
import matplotlib.pyplot as plt
import re

# Diretório onde as partições estão salvas
base_dir = "../Base de Dados/HAREM/divisions/second/iterative/"

number_divisions = 10

# Inicializar lista para as classes teóricas
unique_lbl_list = []

# Lista de abreviações
abbreviations = r"\b(?:Sr|S|Sra|Dr|Prof|Eng|Mme|Mestre|Vº|Vª)\.$"  # Abreviações que não devem terminar uma sentença
pattern = r"[.!?](\s|$)"  # Verifica ponto, exclamação ou interrogação seguidos de espaço ou fim da linha

# Contadores gerais
total_counts = Counter()
total_sentences_por_div = {}
total_tokens_por_div = {}

for file_name in sorted(os.listdir(base_dir)):
    if file_name.startswith("division_"):
        partition_path = os.path.join(base_dir, file_name)
        with open(partition_path, 'r', encoding='utf-8') as file:
            sentence_count = 0
            token_count = 0
            for line in file:
                line = line.strip()
                if line:
                    token_data = line.split()
                    label = token_data[-1]
                    token_count += 1  # Contando tokens
                    if label.startswith("B-"):  # Considerar apenas os rótulos com prefixo 'B-'
                        simplified_label = label.split("-")[-1]
                        unique_lbl_list.append(simplified_label)
                        total_counts[simplified_label] += 1
                    
                    # Verificar se a linha termina com ponto, exclamação ou interrogação
                    # e garantir que não seja uma abreviação
                    if re.search(pattern, line) and not re.search(abbreviations, line):
                        sentence_count += 1
            
            total_sentences_por_div[file_name] = sentence_count
            total_tokens_por_div[file_name] = token_count

# Obter classes teóricas únicas e ordenadas
classes_teoricas = sorted(set(unique_lbl_list))

# Calcular os valores teóricos dividindo as contagens totais por número de divisões
valores_teoricos = [total_counts[cls] / number_divisions for cls in classes_teoricas]

# Inicializando dicionários para armazenar contagens por partição
qtd_classes_por_div = {}
total_entidades_por_div = {}

# Iterar novamente pelas partições para calcular os valores reais
for file_name in sorted(os.listdir(base_dir)):
    if file_name.startswith("division_"):
        partition_path = os.path.join(base_dir, file_name)

        # Inicializar contador para esta partição
        labels_count = Counter()

        with open(partition_path, 'r', encoding='utf-8') as file:
            for line in file:
                line = line.strip()
                if line:
                    token_data = line.split()
                    label = token_data[-1]
                    if label.startswith("B-"):  # Considerar apenas os rótulos com prefixo 'B-'
                        simplified_label = label.split("-")[-1]
                        labels_count[simplified_label] += 1

        # Armazenar as contagens para a partição atual
        qtd_classes_por_div[file_name] = labels_count
        total_entidades_por_div[file_name] = sum(labels_count.values())

# Calculando o total geral de todas as partições
total_entidades_geral = sum(total_entidades_por_div.values())
total_sentences_geral = sum(total_sentences_por_div.values())
total_tokens_geral = sum(total_tokens_por_div.values())

# Exibindo os totais por partição e o total geral
print("Totais de entidades, sentenças e tokens por partição:")
for div_name in total_entidades_por_div:
    print(f"  {div_name}: {total_entidades_por_div[div_name]} entidades, "
          f"{total_sentences_por_div[div_name]} sentenças, "
          f"{total_tokens_por_div[div_name]} tokens")
print(f"\nTotal geral de entidades: {total_entidades_geral}")
print(f"Total geral de sentenças: {total_sentences_geral}")
print(f"Total geral de tokens: {total_tokens_geral}\n")

# Gerando os gráficos para cada partição
for div_name, labels_count in qtd_classes_por_div.items():
    # Ordenar as contagens conforme as classes teóricas
    qtd = [labels_count.get(cls, 0) for cls in classes_teoricas]

    plt.figure(figsize=(10, 6))
    bars_real = plt.barh(classes_teoricas, qtd, color="pink", label="Contagem Real")
    bars_teorico = plt.barh(classes_teoricas, valores_teoricos, color="lightblue", alpha=0.5, label="Valor Teórico")

    # Adicionando os valores nas barras
    for i, count in enumerate(qtd):
        plt.text(count, i, str(count), va="center_baseline", ha="right", fontsize=10)

    for i, teorico in enumerate(valores_teoricos):
        if teorico > 0:
            plt.text(teorico, i, f"{teorico:.1f}", va="center_baseline", ha="right", fontsize=10, color="blue")

    plt.title(f"Entities per Class in Partition: {div_name}", fontsize=15)
    plt.xlabel("Number of Entities", fontweight="bold", fontsize=12)
    plt.ylabel("Classes", fontweight="bold", fontsize=12)

    plt.legend()

    # Salvando o gráfico
    output_path = os.path.join(base_dir, f"entities_per_class_{div_name}.png")
    plt.savefig(output_path, dpi=300, format="png", bbox_inches="tight")
    print(f"Gráfico salvo: {output_path}")

print("Todos os gráficos foram gerados e salvos com sucesso!")


ModuleNotFoundError: No module named 'matplotlib'